In [ ]:
''' 임시로 엑셀 변환하는 코드 '''
# import os
# import pandas as pd
# # 변환할 CSV 파일의 경로와 이름
# csv_file = "../dataset/본문_형태소분석/2018산림복지_본문_형태소분석(형용사).csv"

# # CSV 파일을 데이터프레임으로 읽기
# dataframe = pd.read_csv(csv_file, encoding='UTF-8')

# # XLSX 파일로 변환
# xlsx_file = os.path.splitext(csv_file)[0] + '.xlsx'
# dataframe.to_excel(xlsx_file, index=False)

In [1]:
import pandas as pd              # pandas 패키지 임포트
import numpy as np               # numpy 패키지 임포트
from konlpy.tag import Okt       # 한글 분석 패키지 konlpy 에서 Okt 형태소 분석기 임포트
from tqdm import tqdm     # for 반복문 진행상황 표시
from collections import Counter
# scikit-learn에서 제공하는 TF-IDF 생성 패키지 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()   # 코드 단순화를 위해 Okt()를 변수 okt에 할당

In [2]:
# 불용어 목록 만들기
# ※ 분석할 문장에서 조사를 뽑아 유일값만 남기면 됨.

df_josa = pd.read_csv("../dataset/2022/본문_형태소분석/3월/나무문화_3월_형태소분석(조사).csv", encoding='UTF-8')
josa_list = df_josa["조사"].tolist() # 이다, 보다는 지워야 함.
josa_list.remove("이다")
josa_list.remove("보다")
print(len(josa_list))

396


In [3]:
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_3월_형태소분석.txt", encoding='UTF-8')
df

,split_str,org_idx,morphs,pos_tag
0,4월 전시회 추천 리움 미술관 순회전 인간 일곱 개의 질문 ft 전남도립미술관 인간...,0,"['4월', '전시회', '추천', '리움', '미술관', '순', '회전', '인...","[('4월', 'Number'), ('전시회', 'Noun'), ('추천', 'No..."
1,그때 너무 가고 싶었는데 어찌나 인기가 많던지 예약이 불가했어요,0,"['그때', '너무', '가고', '싶었는데', '어찌나', '인기', '가', '...","[('그때', 'Noun'), ('너무', 'Adverb'), ('가다', 'Ver..."
2,전시 마지막날까지 전일매진 을 기록하는 뜨거운 반응을 보였는데요 그 전시가 이곳 전...,0,"['전시', '마지막', '날', '까지', '전일', '매진', '을', '기록'...","[('전시', 'Noun'), ('마지막', 'Noun'), ('날', 'Noun'..."
3,그 전시를 오늘은 일을 핑계로 이렇게 편히 와서 봤네요,0,"['그', '전시', '를', '오늘', '은', '일', '을', '핑계', '로...","[('그', 'Noun'), ('전시', 'Noun'), ('를', 'Josa'),..."
4,전시명 인간 일곱 개의 질문 장소 전남도립미술관 주소 전남 광양시 광양읍 순광로 6...,0,"['전시', '명', '인간', '일곱', '개', '의', '질문', '장소', ...","[('전시', 'Noun'), ('명', 'Suffix'), ('인간', 'Noun..."
...,...,...,...,...
802678,엑실레스 요새 Forte di Exilles 카살레 몬페라토 또한 인상적인 방어 체...,16127,"['엑실레스', '요새', 'Forte', 'di', 'Exilles', '카', ...","[('엑실레스', 'Noun'), ('요새', 'Noun'), ('Forte', '..."
802679,피네롤로 또한 요새화된 중요한 성채로 지리적 중요성으로 인하여 17세기 동안 프랑스...,16127,"['피', '네롤', '로', '또한', '요새화', '된', '중요한', '성채'...","[('피', 'Noun'), ('네롤', 'Noun'), ('로', 'Josa'),..."
802680,베루아 사보이아 Verrua Savoia 의 요새는 복잡한 성벽을 갖추고 있어 17...,16127,"['베', '루아', '사보이아', 'Verrua', 'Savoia', '의', '...","[('베', 'Noun'), ('루아', 'Noun'), ('사보이아', 'Noun..."
802681,니차 또한 바다로 나가는 유일한 출구라는 중요성으로 인하여 요새화되었다,16127,"['니', '차', '또한', '바다로', '나가는', '유일한', '출구', '라...","[('니', 'Modifier'), ('차', 'Noun'), ('또한', 'Nou..."


In [4]:
df['joined_morphs'] = df['morphs'].apply(lambda x: ' '.join(x.strip("]['").split("', '")))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs
0,4월 전시회 추천 리움 미술관 순회전 인간 일곱 개의 질문 ft 전남도립미술관 인간...,0,"['4월', '전시회', '추천', '리움', '미술관', '순', '회전', '인...","[('4월', 'Number'), ('전시회', 'Noun'), ('추천', 'No...",4월 전시회 추천 리움 미술관 순 회전 인간 일곱 개 의 질문 ft 전남 도립 미술...
1,그때 너무 가고 싶었는데 어찌나 인기가 많던지 예약이 불가했어요,0,"['그때', '너무', '가고', '싶었는데', '어찌나', '인기', '가', '...","[('그때', 'Noun'), ('너무', 'Adverb'), ('가다', 'Ver...",그때 너무 가고 싶었는데 어찌나 인기 가 많던지 예약 이 불가했어요
2,전시 마지막날까지 전일매진 을 기록하는 뜨거운 반응을 보였는데요 그 전시가 이곳 전...,0,"['전시', '마지막', '날', '까지', '전일', '매진', '을', '기록'...","[('전시', 'Noun'), ('마지막', 'Noun'), ('날', 'Noun'...",전시 마지막 날 까지 전일 매진 을 기록 하는 뜨거운 반응 을 보였는데요 그 전시 ...
3,그 전시를 오늘은 일을 핑계로 이렇게 편히 와서 봤네요,0,"['그', '전시', '를', '오늘', '은', '일', '을', '핑계', '로...","[('그', 'Noun'), ('전시', 'Noun'), ('를', 'Josa'),...",그 전시 를 오늘 은 일 을 핑계 로 이렇게 편히 와서 봤네요
4,전시명 인간 일곱 개의 질문 장소 전남도립미술관 주소 전남 광양시 광양읍 순광로 6...,0,"['전시', '명', '인간', '일곱', '개', '의', '질문', '장소', ...","[('전시', 'Noun'), ('명', 'Suffix'), ('인간', 'Noun...",전시 명 인간 일곱 개 의 질문 장소 전남 도립 미술관 주소 전남 광양시 광양읍 순...
...,...,...,...,...,...
802678,엑실레스 요새 Forte di Exilles 카살레 몬페라토 또한 인상적인 방어 체...,16127,"['엑실레스', '요새', 'Forte', 'di', 'Exilles', '카', ...","[('엑실레스', 'Noun'), ('요새', 'Noun'), ('Forte', '...",엑실레스 요새 Forte di Exilles 카 살레 몬페라토 또한 인상 적 인 방...
802679,피네롤로 또한 요새화된 중요한 성채로 지리적 중요성으로 인하여 17세기 동안 프랑스...,16127,"['피', '네롤', '로', '또한', '요새화', '된', '중요한', '성채'...","[('피', 'Noun'), ('네롤', 'Noun'), ('로', 'Josa'),...",피 네롤 로 또한 요새화 된 중요한 성채 로 지리 적 중요성 으로 인하여 17 세기...
802680,베루아 사보이아 Verrua Savoia 의 요새는 복잡한 성벽을 갖추고 있어 17...,16127,"['베', '루아', '사보이아', 'Verrua', 'Savoia', '의', '...","[('베', 'Noun'), ('루아', 'Noun'), ('사보이아', 'Noun...",베 루아 사보이아 Verrua Savoia 의 요새 는 복잡한 성벽 을 갖추고 있어...
802681,니차 또한 바다로 나가는 유일한 출구라는 중요성으로 인하여 요새화되었다,16127,"['니', '차', '또한', '바다로', '나가는', '유일한', '출구', '라...","[('니', 'Modifier'), ('차', 'Noun'), ('또한', 'Nou...",니 차 또한 바다로 나가는 유일한 출구 라는 중요성 으로 인하여 요새화 되었다


In [5]:
df['joined_sentences'] = df.groupby('org_idx')['joined_morphs'].transform(lambda x: ' '.join(x))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs,joined_sentences
0,4월 전시회 추천 리움 미술관 순회전 인간 일곱 개의 질문 ft 전남도립미술관 인간...,0,"['4월', '전시회', '추천', '리움', '미술관', '순', '회전', '인...","[('4월', 'Number'), ('전시회', 'Noun'), ('추천', 'No...",4월 전시회 추천 리움 미술관 순 회전 인간 일곱 개 의 질문 ft 전남 도립 미술...,4월 전시회 추천 리움 미술관 순 회전 인간 일곱 개 의 질문 ft 전남 도립 미술...
1,그때 너무 가고 싶었는데 어찌나 인기가 많던지 예약이 불가했어요,0,"['그때', '너무', '가고', '싶었는데', '어찌나', '인기', '가', '...","[('그때', 'Noun'), ('너무', 'Adverb'), ('가다', 'Ver...",그때 너무 가고 싶었는데 어찌나 인기 가 많던지 예약 이 불가했어요,4월 전시회 추천 리움 미술관 순 회전 인간 일곱 개 의 질문 ft 전남 도립 미술...
2,전시 마지막날까지 전일매진 을 기록하는 뜨거운 반응을 보였는데요 그 전시가 이곳 전...,0,"['전시', '마지막', '날', '까지', '전일', '매진', '을', '기록'...","[('전시', 'Noun'), ('마지막', 'Noun'), ('날', 'Noun'...",전시 마지막 날 까지 전일 매진 을 기록 하는 뜨거운 반응 을 보였는데요 그 전시 ...,4월 전시회 추천 리움 미술관 순 회전 인간 일곱 개 의 질문 ft 전남 도립 미술...
3,그 전시를 오늘은 일을 핑계로 이렇게 편히 와서 봤네요,0,"['그', '전시', '를', '오늘', '은', '일', '을', '핑계', '로...","[('그', 'Noun'), ('전시', 'Noun'), ('를', 'Josa'),...",그 전시 를 오늘 은 일 을 핑계 로 이렇게 편히 와서 봤네요,4월 전시회 추천 리움 미술관 순 회전 인간 일곱 개 의 질문 ft 전남 도립 미술...
4,전시명 인간 일곱 개의 질문 장소 전남도립미술관 주소 전남 광양시 광양읍 순광로 6...,0,"['전시', '명', '인간', '일곱', '개', '의', '질문', '장소', ...","[('전시', 'Noun'), ('명', 'Suffix'), ('인간', 'Noun...",전시 명 인간 일곱 개 의 질문 장소 전남 도립 미술관 주소 전남 광양시 광양읍 순...,4월 전시회 추천 리움 미술관 순 회전 인간 일곱 개 의 질문 ft 전남 도립 미술...
...,...,...,...,...,...,...
802678,엑실레스 요새 Forte di Exilles 카살레 몬페라토 또한 인상적인 방어 체...,16127,"['엑실레스', '요새', 'Forte', 'di', 'Exilles', '카', ...","[('엑실레스', 'Noun'), ('요새', 'Noun'), ('Forte', '...",엑실레스 요새 Forte di Exilles 카 살레 몬페라토 또한 인상 적 인 방...,블로그 이탈리아 사보이아 가문 48 개 의 글 이탈리아 사보이아 가문 목록 열기 사...
802679,피네롤로 또한 요새화된 중요한 성채로 지리적 중요성으로 인하여 17세기 동안 프랑스...,16127,"['피', '네롤', '로', '또한', '요새화', '된', '중요한', '성채'...","[('피', 'Noun'), ('네롤', 'Noun'), ('로', 'Josa'),...",피 네롤 로 또한 요새화 된 중요한 성채 로 지리 적 중요성 으로 인하여 17 세기...,블로그 이탈리아 사보이아 가문 48 개 의 글 이탈리아 사보이아 가문 목록 열기 사...
802680,베루아 사보이아 Verrua Savoia 의 요새는 복잡한 성벽을 갖추고 있어 17...,16127,"['베', '루아', '사보이아', 'Verrua', 'Savoia', '의', '...","[('베', 'Noun'), ('루아', 'Noun'), ('사보이아', 'Noun...",베 루아 사보이아 Verrua Savoia 의 요새 는 복잡한 성벽 을 갖추고 있어...,블로그 이탈리아 사보이아 가문 48 개 의 글 이탈리아 사보이아 가문 목록 열기 사...
802681,니차 또한 바다로 나가는 유일한 출구라는 중요성으로 인하여 요새화되었다,16127,"['니', '차', '또한', '바다로', '나가는', '유일한', '출구', '라...","[('니', 'Modifier'), ('차', 'Noun'), ('또한', 'Nou...",니 차 또한 바다로 나가는 유일한 출구 라는 중요성 으로 인하여 요새화 되었다,블로그 이탈리아 사보이아 가문 48 개 의 글 이탈리아 사보이아 가문 목록 열기 사...


In [6]:
df_tf = df[['org_idx', 'joined_sentences']].drop_duplicates()
df_tf.reset_index(drop=True, inplace=True)
df_tf

,org_idx,joined_sentences
0,0,4월 전시회 추천 리움 미술관 순 회전 인간 일곱 개 의 질문 ft 전남 도립 미술...
1,1,브런치 먹으러 헤 이러 스틱 빵 은 별로 끌리는게 없어서 패스 공간 도 넓고 분위기...
2,2,세계사 를 바꾼 10 가지 감염병 저자 조지 무쇼 출판 사람과나무 사이 발매 202...
3,3,길가 에 벚꽃 이 엄청 엄청 많이 피었다 3월 이 끝나가고 4월 이 오면서 벚꽃 이...
4,4,움짤 집착 러 의 데이터 잡아먹는 블로그 주의 밀린 일상 의 정리 feat 쿠사츠 ...
...,...,...
16121,16123,먹은 메뉴 런치 오 마카 세 소비 비용 100 000원 인 당 50 000 광교 에...
16122,16124,blog 발해 43 개 의 글 발해 목록 닫기 발해 글 목록 글 제목 작성 일 이정...
16123,16125,블로그 이탈리아 사보이아 가문 48 개 의 글 이탈리아 사보이아 가문 목록 열기 이...
16124,16126,평창 영월 백덕산 등산 코스 백덕산 1 350 m 블랙 야크 100 대 명산 67 ...


In [7]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = df_tf['joined_sentences'].to_list()

In [8]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(stop_words = josa_list, token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)

In [9]:
tdm.shape

(16126, 413968)

In [10]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.        , 0.03849672, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.1639276 , 0.06647687, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.00144435, 0.00136668, 0.03252983, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01908571, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00087725, 0.00083008, 0.01975757, ..., 0.        , 0.        ,
        0.        ]])

In [11]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)
tfidf_DF

,0,1,2,3,4,5,6,7,8,9,...,413958,413959,413960,413961,413962,413963,413964,413965,413966,413967
0,0.000000,0.038497,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.163928,0.066477,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.036744,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.007569,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16121,0.000000,0.043622,0.013844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16122,0.003142,0.002973,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16123,0.001444,0.001367,0.032530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16124,0.000000,0.019086,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [13]:
tfidf_DF

,0,00,000,0001,00010,0001145380,00012,000123091,0001509157,0001509298,...,힝,힝겐,힝구,힝깅트립,힝로호,힝스,힝헝,힝힝,힢,힣
0,0.000000,0.038497,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.163928,0.066477,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.036744,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.007569,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16121,0.000000,0.043622,0.013844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16122,0.003142,0.002973,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16123,0.001444,0.001367,0.032530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16124,0.000000,0.019086,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
tfidf_DF.to_csv("../dataset/2022/본문_TF-IDF/3월/나무문화_3월_TF-IDF행렬(원본).csv", index=False, encoding='UTF-8')

In [14]:
import re
# 데이터프레임에서 한글로만 이루어진 column 필터링을 위한 정규표현식 패턴
pattern = r'^[ㄱ-ㅎㅏ-ㅣ가-힣\s]+$'

# 한글로만 이루어진 column으로 된 데이터프레임 생성
korean_df = tfidf_DF[[column for column in tfidf_DF.columns if re.match(pattern, column)]]

print(korean_df)

         ㄱ   ㄱㄱ  ㄱㄱㄱ   ㄱㄴ  ㄱㄴㄱㄴ  ㄱㄴㄷ  ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅍㅌㅎㅎㅎ  ㄱㄴㄷㅁ   ㄱㄷ   ㄱㄹ  ...  \
0      0.0  0.0  0.0  0.0   0.0  0.0               0.0   0.0  0.0  0.0  ...   
1      0.0  0.0  0.0  0.0   0.0  0.0               0.0   0.0  0.0  0.0  ...   
2      0.0  0.0  0.0  0.0   0.0  0.0               0.0   0.0  0.0  0.0  ...   
3      0.0  0.0  0.0  0.0   0.0  0.0               0.0   0.0  0.0  0.0  ...   
4      0.0  0.0  0.0  0.0   0.0  0.0               0.0   0.0  0.0  0.0  ...   
...    ...  ...  ...  ...   ...  ...               ...   ...  ...  ...  ...   
16121  0.0  0.0  0.0  0.0   0.0  0.0               0.0   0.0  0.0  0.0  ...   
16122  0.0  0.0  0.0  0.0   0.0  0.0               0.0   0.0  0.0  0.0  ...   
16123  0.0  0.0  0.0  0.0   0.0  0.0               0.0   0.0  0.0  0.0  ...   
16124  0.0  0.0  0.0  0.0   0.0  0.0               0.0   0.0  0.0  0.0  ...   
16125  0.0  0.0  0.0  0.0   0.0  0.0               0.0   0.0  0.0  0.0  ...   

         힝   힝겐   힝구  힝깅트립  힝로호   힝스   힝헝   힝힝    힢

In [15]:
# 각 단어에 대한 TF-IDF의 합
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_count

,단어,tf-idf
0,0,75.859228
1,00,190.531516
2,000,30.071835
3,0001,0.605280
4,00010,0.020183
...,...,...
413963,힝스,0.002953
413964,힝헝,0.047563
413965,힝힝,0.056690
413966,힢,0.024956


In [16]:
# 단어 - TF-IDF 파일로 저장.
word_count.to_csv("../dataset/2022/본문_TF-IDF/3월/나무문화_3월_TF-IDF행렬합.txt", index=False, encoding='UTF-8')